In [ ]:
import os
import sys
from datetime import datetime
sys.path.insert(0, "../../..")
sys.path.insert(0, "../../../../acquire")

from HUGS.Processing import search
from HUGS.Client import Process, Search, Retrieve
from Acquire.ObjectStore import datetime_to_string
from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

from HUGS.Client import Search
from HUGS.Util import get_datapath, load_hugs_json

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from cartopy.feature import BORDERS
import cartopy.crs as ccrs
import matplotlib.cm as cm
import xarray as xr
import numpy as np
import json

import ipyleaflet as ipl
import ipywidgets as ipw

from pathlib import Path

# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

In [ ]:
center = [51.506815, -0.127710]
zoom = 12

# url = "https://api.mapbox.com/styles/v1/hugs/ck5mpl49v0ng71iokprtni7ra/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiaHVncyIsImEiOiJjazVtbzdxd20wNDFjM21wZDFtZzU1ZG5jIn0.qILQ7iSoXbV78pmZZF8Lcg"
url = "https://api.mapbox.com/styles/v1/hugs/ck5ms14cu3jzo1imv5iizbgz8/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiaHVncyIsImEiOiJjazVtbzdxd20wNDFjM21wZDFtZzU1ZG5jIn0.qILQ7iSoXbV78pmZZF8Lcg"

mapbox_map = ipl.TileLayer(url=url, opacity=1)
map_london = ipl.Map(center=center, zoom=zoom)
map_london.add_layer(mapbox_map)


marker_legend = ipw.HTML(value="<img src='marker-icon-red.png'>   Current site<br><img src='marker-icon-red.png'>   Future site")
marker_control = ipl.WidgetControl(widget=marker_legend, position="topright")
# map_london.add_control(marker_control)

lghg_data = load_hugs_json("lghg_sites.json")

for site in lghg_data["current"]:
    curr_site = lghg_data["current"][site]
    
    lat = curr_site["latitude"]
    long = curr_site["longitude"]
    site_name = curr_site["long_name"]
    
    marker = ipl.Marker(location=(lat, long), draggable=False)
    marker.popup = ipw.HTML(value=f"{site_name} ({site})")
    map_london.add_layer(marker)
    
for site in lghg_data["future"]:
    fut_site = lghg_data["future"][site]
    
    lat = fut_site["latitude"]
    long = fut_site["longitude"]
    site_name = fut_site["long_name"]
    
    # Here we want a green icon
    icon = ipl.Icon(icon_url='marker-icon-red.png', icon_size=[25, 40], icon_anchor=[12,15])
    
    marker = ipl.Marker(location=(lat, long), draggable=False, icon=icon)
    marker.popup = ipw.HTML(value=f"{site_name} ({site})")
    map_london.add_layer(marker)
    
london_footprint = "tmb_canopy_footprint_60op.png"

footprint = ipl.ImageOverlay(url=london_footprint, bounds=((50.8735, -1.25429), (51.5867, 0.1103)))
map_london.add_layer(footprint)
    
    
map_london

Map(center=[51.506815, -0.12771], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [ ]:
%ls

#### Figure 4. Map of London showing sites that are currently part (blue markers) and are soon to be part (red markers) of the London GreeHouse Gas network run by the Atmospheric Chemistry Research Group at the University of Bristol. Overlain on the map is data showing tree density / canopy cover across the boroughs of London.

In [ ]:
filepath = "/home/gar/Documents/Devel/hugs/raw_data/TMB-10magl_HR_UKV_EUROPE_201907.nc"

filepath = "/home/gar/Documents/Devel/hugs/raw_data/london_footprints/HR/TMB-10magl_HR_UKV_EUROPE_201906.nc"

ds = xr.open_dataset(filepath)

domain = "EUROPE"

fp_name = "fp_high"
lon_name = "lon_high"
lat_name = "lat_high"

long_values = ds[fp_name][lon_name]
lat_values = ds[fp_name][lat_name]
zero_values = ds[fp_name][:, :, 0]

zero_mod = zero_values#.where(zero_values > 0.000001)

fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
# ax = fig.add_subplot(111, projection=ccrs.Mercator(), frameon=False)

ax.add_feature(BORDERS, edgecolor="0.5")

# ax.set_extent((-0.5,0.3, 51,52),crs=ccrs.PlateCarree())

bigMagma = cm.get_cmap('magma', 512)
lowMagma = ListedColormap(bigMagma(np.linspace(0.1, 0.8, 256)))

levels = np.linspace(np.percentile(ds[fp_name].values, 5), np.percentile(ds[fp_name].values, 99.5), 20)

p = ax.contourf(long_values, lat_values, zero_mod, levels=levels, cmap=lowMagma, extend="both")
# p.set_clim(zero_mod.min(), zero_mod.max())

# ax.coastlines(color="white")

plt.savefig(fname="tmb_canopy_footprint.png", dpi=600, pad_inches=0)